
<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=10>
    Deep Learning - HW4 <br>
<font color=2565AE size=5>
    Electrical Engineering Department <br>
    winter 2024<br>
<font color=3C99D size=5>
    Practical Assignment 3 <br>
<font color=696880 size=4>
    Amirabbas Afzali 

____

# Personal Data

In [1]:
# Set your student number
student_number = '401110437'
Name = 'Parsa'
Last_Name = 'Ghezelbash'

# Rules
- Make sure that all of your cells can be run perfectly. 
- Try to minimize your use of ChatGPT (or any other AI assistant) as much as possible.
- You must create a report for this task in PDF format and explain the main results.

---

## **Introduction**

Large Language Models (LLMs) are a class of deep learning models designed for processing and generating natural language. These models are trained using large amounts of textual data and utilize architectures based on transformers. Some of the applications of these models include text generation, machine translation, text summarization, question answering, and text classification.

### *Encoder-Decoder LLMs*

One of the common architectures in large language models is the Encoder-Decoder architecture. In this architecture, the encoder processes an input sequence and maps it to a latent space. Then, the decoder uses this latent space to generate an output sequence. Models like T5 [1] (Text-to-Text Transfer Transformer) use this architecture to perform various tasks. In T5, all tasks are expressed in a "text-to-text" format, meaning both input and output are text. This model has capabilities such as translation, summarization, and text classification. One of the advantages of the Encoder-Decoder architecture is that it allows the encoder to utilize information from both before and after a word to gain a more comprehensive understanding of the text.

### *Decoder-only LLMs*

Decoder-only models, such as GPT-2, GPT-3, and LLaMA [2], unlike the Encoder-Decoder architecture, only use the decoder part. These models use an autoregressive mode, meaning they predict the next token based on previous tokens. These models are highly efficient for text generation and have found widespread applications today.

Advantages of Decoder-only Models

- Efficiency: Decoder-only models are more efficient than Encoder-Decoder models due to the absence of an independent encoder. This makes them require fewer computational and memory resources.
- Simplicity: Due to their autoregressive nature, these models can easily generate sequences in order.
- Scalability: Due to their simpler architecture, these models can be scaled to much larger sizes.


However, one of the drawbacks of these models is that they can only utilize information from tokens before the current token and cannot use tokens that come after for prediction. This limitation is significant in tasks like classification or translation, where a full understanding of the sequence is needed.



## **Objective of the Exercise**


In this exercise, the goal is to convert a generative Decoder-only language model into an encoder and evaluate its performance on a binary sentiment classification task. The main aim is to modify the Decoder-only model so that it can function as an encoder and better handle tasks requiring bidirectional understanding.

## **In this exercise, you should:**


### In this exercise, you should:

1. **Import a Decoder-only model** and load the weights of a pre-trained version of the model.
2. **Generate several outputs from the model**, and include at most 10 sample outputs in your report for different inputs.  
   You should also briefly explain the effects of key configurations in text generation, including:  
   - `Temperature`
   - `top_k`
   - `top_p`
   - `repetition_penalty`
   - `num_beams`
   - `no_repeat_ngram_size`
3. **Load the SST-2 dataset**, which is part of the GLUE benchmark for sentiment classification.  
   - Note that the model’s output depends on the number of input tokens. 
   - Apply necessary padding to the dataset after loading it to allow for parallel execution of the model.
4. **Remove the model’s final layer**, which outputs to the size of the model’s dictionary.  
   - Use the embedding vector of the first token (CLS token) for classification.
5. As observed in the previous step, sometimes the embedding vector of the first token does not provide a good representation of the entire input text.  
   - **Add a linear layer** with the same input and output dimensions on top of the encoder's output, and use the output of this linear layer (corresponding to the CLS token) for classification.  
   - This step aggregates information of different tokens to get a comprehensive understanding of the input text.
6. **Instead of the linear layer** in the previous section, use a **bidirectional attention layer** with a custom number of heads (preferably 12).
7. **Repeat step 6** using **left-to-right unidirectional attention** and **right-to-left unidirectional attention**.
8. **Load a pre-trained decoder** (preferably BERT-base) and report its **zero-shot performance** (i.e., without needing to train the model) on the test data.

## **Evaluation:**

In this exercise, for each of sections 4, 5, 6, 7, and 8, you need to plot the confusion matrix corresponding to the model's performance on the test data. Additionally, you should plot two separate graphs showing the training loss and the accuracy of the trained models, and compare them with each other, providing an appropriate analysis of your results. Also, note that high accuracy is not expected for sections 4 and 5, but the correctness of your code will be checked. However, for sections 6 and 7, higher accuracy (around 90%) is expected.













## **Let's go:**


Load `gpt2` model:

In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Model, GenerationConfig
import torch
from torch import nn

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

Load `sst-2` dataset:

In [3]:
from datasets import load_dataset

# Load the SST-2 dataset from Hugging Face 
dataset = load_dataset("glue", "sst2")

go ahead:

## 2)

In [6]:
tokenizer.add_special_tokens({"cls_token": "<|CLS|>"})
model.resize_token_embeddings(len(tokenizer), mean_resizing=False)
model.to(device).eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50258, bias=False)
)

In [7]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

prompts = [
    "Once upon a time",
    "Deep learning is all about",
    "In a shocking finding today, scientists discovered",
    "The meaning of life is",
    "The best way to learn is",
    "The most important thing in the world is",
    "The most beautiful thing in the world is",
    "The most dangerous thing in the world is",
    "The most exciting thing in the world is",
    "The most boring thing in the world is",
]

generation_configs = [
    GenerationConfig(
        max_new_tokens=30,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        pad_token_id = tokenizer.pad_token_id,
        num_beams=1,
        no_repeat_ngram_size=2,
        do_sample=True
    ),
]

for i, prompt in enumerate(prompts[:10]):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    print(f"\nPrompt {i+1}: {prompt}\n{'-'*40}")

    for j, gen_cfg in enumerate(generation_configs):
        output_ids = model.generate(
            input_ids,
            generation_config=gen_cfg
        )
        generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        print(f"  [Gen config {j+1}] --> {generated_text}")



Prompt 1: Once upon a time
----------------------------------------


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
c:\Users\Password Please\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:545: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


  [Gen config 1] --> Once upon a time the result for an person to be in this world, and that is

Prompt 2: Deep learning is all about
----------------------------------------
  [Gen config 1] --> Deep learning is all about and. has the most important feature of human behavior, so it should

Prompt 3: In a shocking finding today, scientists discovered
----------------------------------------
  [Gen config 1] --> In a shocking finding today, scientists discovered were the are they it them as which was their and that how is

Prompt 4: The meaning of life is
----------------------------------------
  [Gen config 1] --> The meaning of life is you the, and that there are two ways to live in.


Prompt 5: The best way to learn is
----------------------------------------
  [Gen config 1] --> The best way to learn is a an . * < > -- the one , if something
 we

Prompt 6: The most important thing in the world is
----------------------------------------
  [Gen config 1] --> The most important thing

## 3)

In [ ]:
from torch.utils.data import DataLoader

def tokenize_function(examples):
    texts = ["<|CLS|> " + t for t in examples["sentence"]]
    return tokenizer(texts, truncation=True, padding="max_length", max_length=32)

gpt_tokenized_dataset = dataset.map(tokenize_function, batched=True)
gpt_tokenized_dataset.set_format(type="torch", columns=["input_ids", "label", "attention_mask"])

train_data = gpt_tokenized_dataset["train"]
val_data   = gpt_tokenized_dataset["validation"]
test_data  = gpt_tokenized_dataset["test"]

print("Train sample:", train_data[0])

train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_data, batch_size=8, num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_data, batch_size=8, num_workers=4, pin_memory=True)

Train sample: {'label': tensor(0), 'input_ids': tensor([50257,  7808,   649,  3200,   507,   422,   262, 21694,  4991,   220,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])}


In [9]:
train_subset_loader = DataLoader(train_data.train_test_split(test_size=0.01, seed=42)["test"], batch_size=8)
val_subset_loader   = DataLoader(val_data.train_test_split(test_size=0.01, seed=42)["test"], batch_size=8)
test_subset_loader  = DataLoader(test_data.train_test_split(test_size=0.01, seed=42)["test"], batch_size=8)

## 4)

In [ ]:
model_without_lm_head = GPT2Model.from_pretrained("gpt2")
model_without_lm_head.resize_token_embeddings(len(tokenizer))
model_without_lm_head.to(device).eval()

def extract_cls_embedding(input_ids, attention_mask):
    outputs = model_without_lm_head(
        input_ids=input_ids,
        attention_mask=attention_mask
    )
    hidden_states = outputs.last_hidden_state
    cls_embed = hidden_states[:, 0, :]
    return cls_embed

sample = train_data[0]
input_ids = sample["input_ids"].unsqueeze(0).to(device)
attention_mask = sample["attention_mask"].unsqueeze(0).to(device)

cls_vector = extract_cls_embedding(input_ids, attention_mask)
print("CLS vector shape:", cls_vector.shape)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


CLS vector shape: torch.Size([1, 768])


## 5)

In [ ]:
class GPT2ClassifierLinear(nn.Module):
    def __init__(self, gpt2_model, hidden_dim=768, num_labels=2):
        super().__init__()
        self.gpt2 = gpt2_model
        self.linear = nn.Linear(hidden_dim, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.gpt2(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state
        cls_token = hidden_states[:, 0, :]
        logits = self.linear(cls_token)
        return logits

model_without_lm_head = GPT2Model.from_pretrained("gpt2")
model_without_lm_head.resize_token_embeddings(len(tokenizer))
model_cls_linear = GPT2ClassifierLinear(model_without_lm_head).to(device)

print("GPT2ClassifierLinear created.")

GPT2ClassifierLinear created.


In [12]:
from tqdm import tqdm

def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    for batch in tqdm(loader, desc="Training"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += (logits.argmax(dim=-1) == labels).sum().item()
        total_samples += labels.size(0)

    return total_loss / len(loader), total_correct / total_samples

def eval_model(model, loader, criterion):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluation"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)

            total_loss += loss.item()
            total_correct += (logits.argmax(dim=-1) == labels).sum().item()
            total_samples += labels.size(0)
    
    return total_loss / len(loader), total_correct / total_samples


In [13]:
from torch.optim import AdamW

optimizer = AdamW(model_cls_linear.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()
n_epochs = 3

for epoch in range(n_epochs):
    train_loss, train_acc = train_epoch(model_cls_linear, train_subset_loader, optimizer, criterion)
    val_loss, val_acc = eval_model(model_cls_linear, val_subset_loader, criterion)
    print(f"Epoch {epoch+1}/{n_epochs} -> Train loss: {train_loss:.4f}, Train acc: {train_acc:.4f}, Val loss: {val_loss:.4f}, Val acc: {val_acc:.4f}")
    
test_loss, test_acc = eval_model(model_cls_linear, val_subset_loader, criterion)
print(f"Val loss: {test_loss:.4f}, Val acc: {test_acc:.4f}")

Evaluation: 100%|██████████| 2/2 [00:00<00:00, 18.65it/s]


Epoch 1/3 -> Train loss: 0.7148, Train acc: 0.5831, Val loss: 0.6098, Val acc: 0.7778


Evaluation: 100%|██████████| 2/2 [00:00<00:00, 13.83it/s]


Epoch 2/3 -> Train loss: 0.7014, Train acc: 0.5267, Val loss: 0.5722, Val acc: 0.7778


Evaluation: 100%|██████████| 2/2 [00:00<00:00, 13.46it/s]


Epoch 3/3 -> Train loss: 0.7008, Train acc: 0.5326, Val loss: 0.6172, Val acc: 0.7778


Evaluation: 100%|██████████| 2/2 [00:00<00:00, 17.59it/s]

Val loss: 0.6172, Val acc: 0.7778


## 6)

In [14]:
class GPT2ClassifierBiAttn(nn.Module):
    def __init__(self, gpt2_model, hidden_dim=768, num_labels=2, num_heads=12):
        super().__init__()
        self.gpt2 = gpt2_model
        self.attn = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=num_heads, batch_first=True)
        self.linear = nn.Linear(hidden_dim, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.gpt2(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state
        
        cls_token = hidden_states[:, 0:1, :]

        out, _ = self.attn(query=cls_token, key=hidden_states, value=hidden_states)

        logits = self.linear(out.squeeze(1))
        return logits

model_without_lm_head = GPT2Model.from_pretrained("gpt2")
model_without_lm_head.resize_token_embeddings(len(tokenizer))
model_cls_biattn = GPT2ClassifierBiAttn(
    gpt2_model=model_without_lm_head, 
    hidden_dim=model_without_lm_head.config.hidden_size, 
    num_labels=2, 
    num_heads=12
).to(device)

print("GPT2ClassifierBiAttn created (bidirectional attention).")

GPT2ClassifierBiAttn created (bidirectional attention).


In [15]:
optimizer = AdamW(model_cls_biattn.parameters(), lr=5e-5)
n_epochs = 3

for epoch in range(n_epochs):
    train_loss, train_acc = train_epoch(model_cls_biattn, train_subset_loader, optimizer, criterion)
    val_loss, val_acc = eval_model(model_cls_biattn, val_subset_loader, criterion)
    print(f"Epoch {epoch+1}/{n_epochs} -> Train loss: {train_loss:.4f}, Train acc: {train_acc:.4f}, Val loss: {val_loss:.4f}, Val acc: {val_acc:.4f}")

test_loss, test_acc = eval_model(model_cls_linear, val_subset_loader, criterion)
print(f"Val loss: {test_loss:.4f}, Val acc: {test_acc:.4f}")

Evaluation: 100%|██████████| 2/2 [00:00<00:00, 13.19it/s]


Epoch 1/3 -> Train loss: 0.7893, Train acc: 0.5786, Val loss: 0.8780, Val acc: 0.3333


Evaluation: 100%|██████████| 2/2 [00:00<00:00, 12.15it/s]


Epoch 2/3 -> Train loss: 0.6967, Train acc: 0.5935, Val loss: 0.7782, Val acc: 0.3333


Evaluation: 100%|██████████| 2/2 [00:00<00:00, 11.87it/s]


Epoch 3/3 -> Train loss: 0.5218, Train acc: 0.7463, Val loss: 0.4636, Val acc: 0.7778


Evaluation: 100%|██████████| 2/2 [00:00<00:00, 14.64it/s]

Val loss: 0.6172, Val acc: 0.7778


## 7)

In [16]:
def generate_causal_mask(seq_len):
    mask = torch.ones(seq_len, seq_len).triu(1).bool()
    return mask

class GPT2ClassifierCausalAttn(nn.Module):
    def __init__(self, gpt2_model, hidden_dim=768, num_labels=2, num_heads=12, l2r=True):
        super().__init__()
        self.gpt2 = gpt2_model
        self.attn = nn.MultiheadAttention(hidden_dim, num_heads, batch_first=True)
        self.linear = nn.Linear(hidden_dim, num_labels)
        self.l2r = l2r

    def forward(self, input_ids, attention_mask):
        outputs = self.gpt2(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state
        B, L, H = hidden_states.shape

        cls_token = hidden_states[:, 0:1, :]
        causal_mask = generate_causal_mask(L).to(hidden_states.device)
        
        if not self.l2r:
            causal_mask = torch.flip(causal_mask, dims=(0,1))
        
        out, _ = self.attn(
            query=cls_token,
            key=hidden_states,
            value=hidden_states,
            attn_mask=causal_mask[:1, :],
        )
        logits = self.linear(out.squeeze(1))
        return logits

model_without_lm_head = GPT2Model.from_pretrained("gpt2")
model_without_lm_head.resize_token_embeddings(len(tokenizer))
model_cls_l2r = GPT2ClassifierCausalAttn(
    model_without_lm_head, 
    hidden_dim=model_without_lm_head.config.hidden_size, 
    num_labels=2,
    num_heads=12,
    l2r=True
).to(device)

model_cls_r2l = GPT2ClassifierCausalAttn(
    model_without_lm_head,
    hidden_dim=model_without_lm_head.config.hidden_size,
    num_labels=2,
    num_heads=12,
    l2r=False
).to(device)

print("GPT2ClassifierCausalAttn created for L2R and R2L unidirectional attention.")

GPT2ClassifierCausalAttn created for L2R and R2L unidirectional attention.


In [17]:
optimizer = AdamW(model_cls_l2r.parameters(), lr=5e-5)
n_epochs = 3

for epoch in range(n_epochs):
    train_loss, train_acc = train_epoch(model_cls_l2r, train_subset_loader, optimizer, criterion)
    val_loss, val_acc = eval_model(model_cls_l2r, val_subset_loader, criterion)
    print(f"Epoch {epoch+1}/{n_epochs} -> Train loss: {train_loss:.4f}, Train acc: {train_acc:.4f}, Val loss: {val_loss:.4f}, Val acc: {val_acc:.4f}")
    
test_loss, test_acc = eval_model(model_cls_linear, val_subset_loader, criterion)
print(f"Val loss: {test_loss:.4f}, Val acc: {test_acc:.4f}")

Evaluation: 100%|██████████| 2/2 [00:00<00:00, 69.15it/s]


Epoch 1/3 -> Train loss: 0.7308, Train acc: 0.5653, Val loss: 0.5712, Val acc: 0.7778


Evaluation: 100%|██████████| 2/2 [00:00<00:00, 58.98it/s]


Epoch 2/3 -> Train loss: 0.6994, Train acc: 0.5727, Val loss: 0.6117, Val acc: 0.7778


Evaluation: 100%|██████████| 2/2 [00:00<00:00, 77.12it/s]


Epoch 3/3 -> Train loss: 0.6903, Train acc: 0.5682, Val loss: 0.5519, Val acc: 0.7778


Evaluation: 100%|██████████| 2/2 [00:00<00:00, 80.22it/s]

Val loss: 0.6172, Val acc: 0.7778


## 8)

In [15]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=128)

bert_tokenized_dataset = dataset.map(tokenize_function, batched=True)

bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from torch.utils.data import DataLoader
from tqdm import tqdm

tokenized_train = bert_tokenized_dataset["train"]
tokenized_val   = bert_tokenized_dataset["validation"]
tokenized_test  = bert_tokenized_dataset["test"]

tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
tokenized_val.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask"])

def collate_fn(batch):
    input_ids = torch.Tensor([item['input_ids'] for item in batch]).long()
    attention_mask = torch.Tensor([item['attention_mask'] for item in batch]).long()
    labels = [item['label'] for item in batch]
    
    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = torch.tensor(labels)
    
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'label': labels}

train_loader = DataLoader(tokenized_train, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(tokenized_val, batch_size=8, collate_fn=collate_fn)
test_loader = DataLoader(tokenized_test, batch_size=8, collate_fn=collate_fn)

correct = 0
total = 0

with torch.no_grad():
    for batch in tqdm(val_loader, desc="Testing"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device) if "label" in batch else None

        outputs = bert_model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = logits.argmax(dim=-1)
        
        total += labels.size(0)
        correct += (predictions == labels).sum().item()
        
print(f"Test accuracy: {correct / total:.4f}")

Testing:   0%|          | 0/109 [00:00<?, ?it/s]


ValueError: only one element tensors can be converted to Python scalars

In [18]:
from transformers import BertTokenizer, BertForSequenceClassification

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)
bert_model.eval()

print("Loaded BERT-base for classification (zero-shot).")

# Evaluate on test set (zero-shot)
correct = 0
total = 0

for example in val_subset_loader:
    input_ids = example['input_ids'].to(device)
    attention_mask = example['attention_mask'].to(device)
    
    with torch.inference_mode():
        logits = bert_model(input_ids, attention_mask=attention_mask).logits  # shape [1, 2]
    pred_label = torch.argmax(logits, dim=-1).item()
    if pred_label == example["label"]:
        correct += 1
    total += 1

accuracy_bert = correct / total
print(f"Zero-shot Test Accuracy with BERT-base: {accuracy_bert:.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded BERT-base for classification (zero-shot).


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


---------
### References

[1] Raffel, Colin, Noam Shazeer, Adam Roberts, et al. (2020). *Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer*. [Link to paper](https://arxiv.org/abs/1910.10683)

[2] Touvron, Hugo, et al. (2023). *LLaMA 2: Open Foundation and Fine-Tuned Chat Models*. [Link to paper](https://arxiv.org/abs/2307.09288)

<span style="color:yellow;">*For further reading on this field of research, you can refer to the following papers:*</span>

[3] BehnamGhader, Adlakha, et al. (2024). *LLM2Vec: Large Language Models Are Secretly Powerful Text Encoders*. [Link to paper](https://arxiv.org/abs/2404.05961)

[4] Gao, Tianyu, et al. (2021). *SimCSE: Simple Contrastive Learning of Sentence Embeddings*. [Link to paper](https://arxiv.org/abs/2104.08821)

[5] Lee, et al. (2023). *NV-Embed: Improved Techniques for Training LLMs as Generalist Embedding Models*. [Link to paper](https://arxiv.org/abs/2405.17428)




# **Best regards.**